In [1]:
import numpy as np
from trust_constr import minimize, NonlinearConstraint, LinearConstraint, Bounds, check_grad

## Example 1: Nonlinear Inequality Constraint
Example 15.1 from [1]

Solve:
$$\min_{x,y} f(x,y)=\frac{1}{2}(x-2)^2+\frac{1}{2}\left(y-\frac{1}{2}\right)^2$$
Subject to:
$$(x+1)^{-1}-y-\frac{1}{4}\ge0$$
$$x\ge0$$
$$x\ge0$$

Solution: $$(x,y) = (1.953, 0.089)$$


First solve without defining gradient (finite difference gradient will be used):

In [2]:
def objective(x):
    return 0.5*(x[0]-2)**2+0.5*(x[1]-0.5)**2

def ineq_constraint(x):
    return 1/(x[0]+1)-x[1]-0.25

# Use np.inf of -np.inf to define a single sided constraint
# If there are more than one constraint, that constraints will 
# be a list containing all of the constraints
constraints = NonlinearConstraint(ineq_constraint, 0, np.inf)

# set bounds on the variables
bounds = Bounds([0,0], [np.inf, np.inf])

# define starting point for optimization
x0 = np.array([5.0, 1.0])

res = minimize(objective, x0, bounds=bounds, constraints=constraints)

print("Solution =", res.x)
print(f"Obtained using {res.nfev} objective function evaluations.")

Solution = [1.95282327 0.08865882]
Obtained using 42 objective function evaluations.


Now define the gradient for objective and constraint and check gradients:

In [3]:
def objective_gradient(x):
    return np.array([(x[0]-2), (x[1]-0.5)])

def ineq_gradient(x):
    return np.array([-1/((x[0]+1)**2), -1])

# check gradients against finite difference gradient
# use 5 random points
for x in np.random.uniform(low=[0,0], high=[10,10], size=(5,2)):
    print("objective difference: ", check_grad(objective, objective_gradient, x))
    print("constraint difference:", check_grad(ineq_constraint, ineq_gradient, x))



objective difference:  3.9742399623855506e-08
constraint difference: 3.0021780755840055e-09
objective difference:  1.2926677659337474e-07
constraint difference: 3.007207191596528e-08
objective difference:  3.9461736525381035e-08
constraint difference: 4.605664752355132e-09
objective difference:  4.1213255921399863e-07
constraint difference: 1.0564694242695682e-07
objective difference:  1.32529331705622e-07
constraint difference: 4.336607009082505e-08


Finally, minimize using the gradient functions that were just test:

In [4]:
constraints = NonlinearConstraint(ineq_constraint, 0, np.inf, jac=ineq_gradient)

res = minimize(objective, x0, jac=objective_gradient, bounds=bounds, constraints=constraints)

print("Solution =", res.x)
print(f"Obtained using {res.nfev} objective function evaluations.")

Solution = [1.95282328 0.08865881]
Obtained using 14 objective function evaluations.


## Example 2: Nonlinear Equality Constraint
Example 15.2 from [1]

Solve:

$$\min_{x,y} x^2+y^2$$

Subject to:

$$(x-1)^3 - y^2 = 0$$

Solution:
$$(x,y)=(1,0)$$



In [5]:
objective2 = lambda x: x[0]**2 + x[1]**2
objective2_gradient = lambda x: np.array([2*x[0], 2*x[1]])

eq_constraint = lambda x: (x[0]-1)**3 - x[1]**2
eq_gradient = lambda x: np.array([3*(x[0]-1)**2, -2*x[1]]) 

# Make the upper and lower bound both zero to define an equality constraint
constraints = NonlinearConstraint(eq_constraint, 0, 0, jac=eq_gradient) 

x0 = np.array([5, 2])

res = minimize(objective2, x0, jac=objective2_gradient, constraints=constraints)

print("Solution =", res.x)
print(f"Obtained using {res.nfev} objective function evaluations.")

Solution = [9.99966899e-01 3.36074169e-09]
Obtained using 181 objective function evaluations.


## Example 3: Linear Constraint
Example 15.3 from [1]

Solve:

$$\min_{\mathbf{x}} sin(x_1+x_2)+x^2_3+\frac{1}{3}\left(x_4+x^4_5+\frac{x_6}{2}\right)$$

Subject to:

$$8x_1-6x_2+x_3+9x_4+4x_5=6$$
$$3x_1+2x_2-x_4+6x_5+4x_6=-4$$

Solution:

$$\mathbf{x}=[]$$

In [6]:
objective3 = lambda x: np.sin(x[0]+x[1]) + x[2]**2+(1/3)*(x[3]+x[4]**4+x[5]/2)

# define the linear constraint matrix
A = np.array([[8, -6, 1, 9, 4, 0],
              [3, 2, 0, -1, 6, 4]])

x0 = np.zeros(6)

res = minimize(objective3, x0, constraints=LinearConstraint(A, [6,-4], [6,-4]))

print("Solution =", res.x)
print(f"Obtained using {res.nfev} objective function evaluations.")

A@res.x

/home/grem/git/trust-constr/trust_constr/_hessian_update_strategy.py:176: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '


Solution = [ 5.44748930e+06 -5.31968457e+06  2.47205015e-01 -8.38866858e+06
  2.82151097e-01 -3.52294326e+06]
Obtained using 7833 objective function evaluations.


array([ 6., -4.])

## References
[1] Nocedal, Jorge, and Stephen J. Wright. *Numerical Optimization*. 2nd ed. Springer Series in Operations Research. New York: Springer, 2006.
